In [1]:
#comment out for local testing out of Main Sequence Platform
import dotenv
dotenv.load_dotenv('../../.env')

from mainsequence import VAM_CONSTANTS
from mainsequence.virtualfundbuilder.portfolio_interface import PortfolioInterface
from mainsequence.virtualfundbuilder.agent_interface import TDAGAgent

2025-03-26T12:50:56.159752Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:39 in refresh_headers())
2025-03-26T12:50:56.677508Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:39 in refresh_headers())
2025-03-26T12:50:56.720458Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:39 in refresh_headers())
2025-03-26T12:50:56.721688Z [debug    ] Requesting GET from http://192.168.178.17:8000/orm/api/assets/api/constants application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:55 in make_request())
2025-03-26T12:50:57.349156Z [debug    ] Requesting GET from http://192.168.178.17:8000/orm/api/pods/projects/get_user_default_project/ application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:55 in make_request())

# Create a new Asset Category for the Portfolio

In [6]:
from mainsequence.client import Asset, AssetCategory, MarketsTimeSeriesDetails

market_cap_source = MarketsTimeSeriesDetails.get(
    unique_identifier=VAM_CONSTANTS.data_sources_constants.HISTORICAL_MARKET_CAP
)

last_obs = market_cap_source.get_last_observation(execution_venue_symbol=VAM_CONSTANTS.ALPACA_EV_SYMBOL)
top50_market_caps = last_obs.sort_values("market_cap", ascending=False).iloc[:50]
top50_assets = Asset.filter(unique_identifier__in=top50_market_caps["unique_identifier"].to_list())

category = AssetCategory.get_or_none(name="Top50USEquities", source="user_defined")
if category is None:
    crypto_category = AssetCategory.create(name="Top50USEquities", source="user_defined", assets=[a.id for a in top50_assets])
else:
    category.update_assets(asset_ids=[a.id for a in top50_assets])

2025-03-26T12:53:35.667500Z [debug    ] Requesting GET from http://192.168.178.17:8000/orm/api/data_sources/markets-time-series-details/ application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:55 in make_request())
2025-03-26T12:53:37.124471Z [debug    ] Requesting POST from http://192.168.178.17:8000/orm/api/ts_manager/local_time_serie/3/get_data_between_dates_from_remote/ application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:55 in make_request())
2025-03-26T12:53:38.030245Z [debug    ] Requesting GET from http://192.168.178.17:8000/orm/api/assets/asset/ application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:55 in make_request())
2025-03-26T12:53:38.673450Z [debug    ] Requesting GET from http://192.168.178.17:8000/orm/api/assets/asset/?asset_type=cash_equity&execution_venue__symbol=alpaca&limit=100&offset=100&symbol__in=HII%2CADBE%2CPRU%2CAKAM%2CWRB%2CRF%2CHLT%2CBAX%2CDGX%2CFE%2CDFS%2CUNP%2CPG%2CIEX%2

# Connect to the TDAG-LLM Agent

In [4]:
tdag_agent = TDAGAgent()

2025-03-26T12:52:36.737044Z [debug    ] Send default documentation to Backend application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at __main__.py:156 in send_default_configuration())
2025-03-26T12:52:36.738758Z [debug    ] Requesting POST from http://192.168.178.17:8000/orm/api/tdag-gpt/register_default_configuration/ application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:55 in make_request())
2025-03-26T12:52:37.238205Z [info     ] Setup TDAG Agent successfull   application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at agent_interface.py:28 in __init__())


# Create a custom Strategy

In [9]:
from mainsequence.virtualfundbuilder.contrib.time_series import MarketCap

portfolio = tdag_agent.generate_portfolio(MarketCap, signal_description="Create me a market cap portfolio using the Top50USEquities asset category.")

portfolio

2025-03-26T12:56:05.327906Z [debug    ] Register signal: MarketCap     application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:675 in _send_strategy_to_registry())
2025-03-26T12:56:05.329590Z [debug    ] Requesting POST from http://192.168.178.17:8000/orm/api/tdag-gpt/register_strategy/ application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:55 in make_request())
2025-03-26T12:56:05.774963Z [info     ] Get configuration for MarketCap ... application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at agent_interface.py:43 in generate_portfolio())
2025-03-26T12:56:05.776621Z [debug    ] Requesting POST from http://192.168.178.17:8000/orm/api/tdag-gpt/create_configuration_for_strategy/ application_name=ms-sdk data_source_id=6 job_run_id=None project_id=7 (at utils.py:55 in make_request())
2025-03-26T12:56:31.010816Z [warning  ] Could not get configuration from TSORM 1 validation error for AssetUniverse
asset_filters.0.execut

# Run the strategy

In [ ]:
res = portfolio.run()
res.head()

2025-03-17T20:04:50.076119Z [debug    ] Requesting GET from http://127.0.0.1:8000/orm/api/data_sources/historical-bars-source/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=3 (at utils.py:55 in make_request())


# Plot the portfolio results

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Create the plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=res.index, 
        y=res["close"], 
        mode="lines+markers",
        name="Portfolio Value",
        line=dict(color='blue', width=1),
        marker=dict(size=2)
    )
)

# Customize layout
fig.update_layout(
    title="Portfolio Value Over Time",
    xaxis_title="Date",
    yaxis_title="Portfolio Value",
    template="plotly_white",
    hovermode="x unified"
)

# Show the plot
fig.show()